In [ ]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "8g",
        "spark.executor.cores": "4",
        "spark.driver.memory": "2g"
    }
}


In [ ]:
# Access configuration
conf = spark.sparkContext.getConf()

# Print relevant executor settings
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Memory:", conf.get("spark.executor.memory"))
print("Executor Cores:", conf.get("spark.executor.cores"))

Starting Spark application


In [1]:
# %%configure -f
# {
#     "conf": {
#         "spark.executor.instances": "4",
#         "spark.executor.memory": "4g",
#         "spark.executor.cores": "2",
#         "spark.driver.memory": "2g"
#     }
# }

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4084,application_1732639283265_4024,pyspark,idle,Link,Link,None,
4087,application_1732639283265_4027,pyspark,idle,Link,Link,None,
4093,application_1732639283265_4033,pyspark,idle,Link,Link,None,
4094,application_1732639283265_4034,pyspark,idle,Link,Link,None,
4104,application_1732639283265_4044,pyspark,idle,Link,Link,None,
4105,application_1732639283265_4045,pyspark,idle,Link,Link,None,
4106,application_1732639283265_4046,pyspark,idle,Link,Link,None,
4108,application_1732639283265_4048,pyspark,idle,Link,Link,None,
4111,application_1732639283265_4051,pyspark,idle,Link,Link,None,
4115,application_1732639283265_4055,pyspark,idle,Link,Link,None,


In [ ]:
# # Access configuration
# conf = spark.sparkContext.getConf()

# # Print relevant executor settings
# print("Executor Instances:", conf.get("spark.executor.instances"))
# print("Executor Memory:", conf.get("spark.executor.memory"))
# print("Executor Cores:", conf.get("spark.executor.cores"))

Starting Spark application


In [1]:
# %%configure -f
# {
#     "conf": {
#         "spark.executor.instances": "8",
#         "spark.executor.memory": "2g",
#         "spark.executor.cores": "1",
#         "spark.driver.memory": "2g"
#     }
# }


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4084,application_1732639283265_4024,pyspark,idle,Link,Link,None,
4087,application_1732639283265_4027,pyspark,idle,Link,Link,None,
4093,application_1732639283265_4033,pyspark,idle,Link,Link,None,
4094,application_1732639283265_4034,pyspark,idle,Link,Link,None,
4104,application_1732639283265_4044,pyspark,idle,Link,Link,None,
4105,application_1732639283265_4045,pyspark,idle,Link,Link,None,
4106,application_1732639283265_4046,pyspark,idle,Link,Link,None,
4108,application_1732639283265_4048,pyspark,idle,Link,Link,None,
4111,application_1732639283265_4051,pyspark,idle,Link,Link,None,
4115,application_1732639283265_4055,pyspark,idle,Link,Link,None,


In [ ]:
# # Access configuration
# conf = spark.sparkContext.getConf()

# # Print relevant executor settings
# print("Executor Instances:", conf.get("spark.executor.instances"))
# print("Executor Memory:", conf.get("spark.executor.memory"))
# print("Executor Cores:", conf.get("spark.executor.cores"))

Starting Spark application


In [ ]:
import time
from sedona.spark import *
from pyspark.sql.functions import col, avg, count
from pyspark.sql import SparkSession

# Start the timer
start_time = time.time()

# Create spark Session
spark = SparkSession.builder \
    .appName("Query 5") \
    .getOrCreate()

sedona = SedonaContext.create(spark)

#Read data from files
crimes_df_2010_2019 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, inferSchema=True)
crimes_df_2020_present = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, inferSchema=True)
stations_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv", header=True, inferSchema=True)

stations_df = stations_df.withColumnRenamed("X", "Station_LON").withColumnRenamed("Y", "Station_LAT")

crimes_df = crimes_df_2010_2019.union(crimes_df_2020_present)

stations_df = stations_df.withColumn("station_point", ST_Point("Station_LON", "Station_LAT"))
crimes_df = crimes_df.withColumn("crime_point", ST_Point("LON", "LAT"))
# stations_df.show()
# crimes_df.show()

crimes_and_stations=crimes_df.crossJoin(stations_df).withColumn("Distance", ST_DistanceSphere("station_point", "crime_point")/1000) # divide with 1000 to conver into km)

crimes_and_stations.createOrReplaceTempView("crimes_and_stations")
final_result = spark.sql("""
    SELECT DIVISION AS `DIVISION`, AVG(Distancea1) AS `Average Distance`, COUNT(DR_NOa1) AS `Number of Crimes`
    FROM (
        SELECT a1.DIVISION AS DIVISION, a1.Distance AS Distancea1 , a2.Distance, a1.DR_NO AS DR_NOa1, a2.DR_NO FROM
        (SELECT DR_NO , MIN(Distance) AS `Distance`
        FROM  crimes_and_stations
        GROUP BY DR_NO) AS `a2`
        INNER JOIN crimes_and_stations AS `a1` 
        WHERE (a1.DR_NO=a2.DR_NO AND a1.Distance=a2.Distance)
        )
    GROUP BY DIVISION
    ORDER BY `Number of Crimes` DESC
""")

# End the timer
end_time = time.time()

final_result.show(n=21, truncate=False)

# End the timer
end_time = time.time()
# Print the execution time
execution_time = end_time - start_time
print(f"Execution Time: {execution_time:.2f} seconds")

spark.stop()